## Importing required libraries

In [35]:
import pandas as pd # library for data analsysis
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import geocoder
import signal
from geopy.geocoders import Nominatim

### Fetching data from the web

Assigning the URl to a variable

In [36]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Now we need to make a get request to obtain the html of the given url

In [37]:
page = urllib.request.urlopen(url)

Using BeautifulSoup, beautify the html content in order to make it more user readable

In [38]:
soup = BeautifulSoup(page)

#### Now we are assuming that there is only one table present on the page, and our data is present in the body of that table alone

So, let us store the contents of the table>tbody tag into a variable

In [39]:
k = soup.table.tbody

Now we need to locate all 'td' which store the actual required values

In [40]:
l = k.find_all('td')

### Creating the dataframe

Let us create an empty dataframe

In [41]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighbourhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,Postal Code,Borough,Neighbourhood


#### Now we need to reetrive the postal code, borough and neighbourhood and append them to our earlier created dataframe

In [42]:
for i in range(0,len(l),3):
    pc = str(l[i].contents)[2:-2]
    bor = ""

    try:
        bor = l[i+1].a.contents[0]
        neigh = l[i+2].a.contents[0]
    except:
        if(bor=='' or bor==None):
            continue
        neigh = str(l[i+2].contents)[2:-4]
    
    if(neigh=='Not assigned'):
        neigh = bor
    neighborhoods = neighborhoods.append({'Postal Code': pc,
                                          'Borough': bor,
                                          'Neighbourhood': neigh}, ignore_index=True)

Let us have a quick look at our data

In [43]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [44]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 208 neighborhoods.


#### Now we need to group (merge) our data, so as show unique postal codes

In [45]:
new_neighbourhood = neighborhoods.groupby('Postal Code').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()

In [46]:
new_neighbourhood

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Let us check the size of our dataframe

In [47]:
new_neighbourhood.shape

(100, 3)

In [14]:
new_neighbourhood['Latitude'] = np.nan
new_neighbourhood['Longitude'] = np.nan

In [16]:
def getCordinates(postal_code):
    while(True):
        try:
            address = '{}, Toronto, Ontario'.format(postal_code)
            geolocator = Nominatim(user_agent="toronto_explorer")
            location = geolocator.geocode(address)
        except:
            pass
        if(location is None):
            pass
        else:
            return location[1]

In [ ]:
i = 0
for postal_code in new_neighbourhood['PostalCode']:
    
    if(postal_code in ext_list):
        continue
    
    try:
        location = getCordinates(postal_code)
        new_neighbourhood['Latitude'][i] = location[0]
        new_neighbourhood['Longitude'][i] = location[1]
        print("Co-ordinates for {} are {}, {}".format(postal_code, location[0], location[1]))
    except:
        print("time out for: "+str(postal_code))
        
    i+=1

In [51]:
new_neighbourhood

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [ ]:
new_neighbourhood